In [1]:
from google.cloud import bigquery

In [2]:
%load_ext google.cloud.bigquery

/home/hyu424/p8_hyu424/venv/lib/python3.12/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


In [6]:
%%bigquery df_1 --project cs544p8-444019
SELECT geo_id
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name='Dane'

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
#q1
ans1 = df_1.iat[0, 0]  
ans1

'55025'

In [8]:
%%bigquery df_2 --project cs544p8-444019
SELECT state_fips_code, COUNT(geo_id) AS counts
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY counts DESC
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

In [16]:
#q2
#df_2 = df_2.set_index("state_fips_code")
result_dict_2 = df_2['counts'].to_dict()
result_dict_2

{'48': 254, '13': 159, '51': 133, '21': 120, '29': 115}

In [17]:
no_cache_dry = bigquery.QueryJobConfig(dry_run=True,use_query_cache=False)
bq = bigquery.Client("cs544p8-444019")
q1 = bq.query("""
SELECT geo_id
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name='Dane'
""", job_config=no_cache_dry)

result_q1=q1.total_bytes_processed / 1024**2

q2=bq.query("""
SELECT state_fips_code, COUNT(geo_id) AS counts
FROM bigquery-public-data.geo_us_boundaries.counties
GROUP BY state_fips_code
ORDER BY counts DESC
LIMIT 5
""", job_config=no_cache_dry)

result_q2=q2.total_bytes_processed / 1024**2

/home/hyu424/p8_hyu424/venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [19]:
#q3
result_q3 = {
    "q1": f"{result_q1}MB",
    "q2": f"{result_q2}MB"
}
result_q3

{'q1': '0.04947185516357422MB', 'q2': '0.03391551971435547MB'}

In [23]:
bq = bigquery.Client("cs544p8-444019")
ds= bigquery.Dataset("cs544p8-444019.p8")
bq.create_dataset(ds, exists_ok=True)

/home/hyu424/p8_hyu424/venv/lib/python3.12/site-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Dataset(DatasetReference('cs544p8-444019', 'p8'))

In [27]:
config = bigquery.ExternalConfig(source_format="PARQUET")
config.source_uris = ["gs://cs544_p8_1688/hdma-wi-2021.parquet"]
# config.autodetect = True
table = bigquery.Table("cs544p8-444019.p8.hdma")
table.external_data_configuration = config
bq.create_table(table, exists_ok=True)

Table(TableReference(DatasetReference('cs544p8-444019', 'p8'), 'hdma'))

In [28]:
#q4
datasets=[]
for ds in bq.list_datasets("cs544p8-444019"):
    datasets.append(ds.dataset_id)
datasets

['p8']

In [38]:
q=bq.query("""
SELECT c.county_name AS county_name, COUNT(h.index) AS application_count 
FROM p8.hdma as h
JOIN bigquery-public-data.geo_us_boundaries.counties AS c
ON h.county_code=c.county_fips_code
GROUP BY c.county_name
ORDER BY application_count DESC
LIMIT 10
""")

In [39]:
df_5=q.to_dataframe()
df_5

,county_name,application_count
0,Milwaukee,46570
1,Dane,38557
2,Waukesha,34159
3,Brown,15615
4,Racine,13007
5,Outagamie,11523
6,Kenosha,10744
7,Washington,10726
8,Rock,9834
9,Winnebago,9310


In [42]:
#q5
#df_5=df_5.set_index('county_name')
result_5=df_5['application_count'].to_dict()
result_5

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}